# URL reference

- Edition: http://www.fifa.com/worldcup/archive/brazil2014/matches/index.html
- Team: http://www.fifa.com/tournaments/archive/worldcup/argentina1978/teams/team=43922/index.html
- Player: http://www.fifa.com/worldcup/archive/brazil2014/teams/index.html
- Player List: http://www.fifa.com//worldcup/archive/edition=2014/library/teams/team=43843/_players/_players_list.html

In [2]:
import os
import re
import hashlib
import requests
import pandas as pd
from lxml.html import parse

if not os.path.exists('.cache'):
    os.makedirs('.cache')

ua = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.116 Safari/537.36'
session = requests.Session()

def get(url):
    '''Return cached lxml tree for url'''
    path = os.path.join('.cache', hashlib.md5(url).hexdigest() + '.html')
    if not os.path.exists(path):
        print url
        response = session.get(url, headers={'User-Agent': ua})
        with open(path, 'w') as fd:
            fd.write(response.text.encode('utf-8'))
    return parse(open(path))

# Editions

In [3]:
editions = [
    'brazil2014',
    'southafrica2010',
    'germany2006',
    'koreajapan2002',
    'france1998',
    'usa1994',
    'italy1990',
    'mexico1986',
    'spain1982',
    'argentina1978',
    'germany1974',
    'mexico1970',
    'england1966',
    'chile1962',
    'sweden1958',
    'switzerland1954',
    'brazil1950',
    'france1938',
    'italy1934',
    'uruguay1930',
]

# Matches

In [4]:
stage_replace = {
  'Group 1': 'Group A',
  'Group 2': 'Group B',
  'Group 3': 'Group C',
  'Group 4': 'Group D',
  'Group 5': 'Group E',
  'Group 6': 'Group F',
  'First round': 'Group A',
  'Preliminary round': 'Group A',
  'Match for third place': 'Third place',
}

In [5]:
matches, team_name, team_code = [], {}, {}
for edition in editions:
    url = 'http://www.fifa.com/worldcup/archive/{:s}/matches/index.html'
    tree = get(url.format(edition))
    for t in tree.xpath('.//div[contains(@class, "col-xs-12 clear-grid")]'):
        venue = t.find_class('mu-i-venue')
        match = t.find_class('mu-i-matchnum')
        date = t.find_class('mu-i-date')
        time = t.find_class('mu-i-datetime')
#         time[0].text_content()[time[0].text_content().index(':')-3:time[0].text_content().index(':')+3]
        stage = t.find_class('mu-i-group')
        flag1 = t.find_class('t home')
        flag2 = t.find_class('t away')
        matchid = t.find_class('mu result')
        resultid = t.find_class('mu-m-link')  
        if venue and match and date and stage and flag1 and flag2 and matchid and resultid and time:
                
            start = resultid[0].attrib['href'].find('und=') + 4
            end = resultid[0].attrib['href'].find('/matc',start)
            
            row = {'Edition': edition,
            'Venue': venue[0].text_content() ,
            'Match': match[0].text_content(),
            'Date': date[0].text_content(),
            'Stage': stage[0].text_content(),
            'Country1': flag1[0].text_content()[-3:],
            'Country2': flag2[0].text_content()[-3:],
            'TeamID1': flag1[0].attrib['data-team-id'],
            'TeamID2': flag2[0].attrib['data-team-id'],
            'RoundID': resultid[0].attrib['href'][start:end],
            'MatchID': matchid[0].attrib['data-id'],
            'Time': time[0].text_content()[time[0].text_content().index(':')-3:time[0].text_content().index(':')+3],
}
            
            team_name[str(flag1[0].text_content()[-3:])] = flag1[0].text_content()[0:-3]
            team_name[str(flag2[0].text_content()[-3:])] = flag2[0].text_content()[0:-3] 
            team_code[str(flag1[0].text_content()[-3:])] = flag1[0].attrib['data-team-id']
            team_code[str(flag2[0].text_content()[-3:])] = flag2[0].attrib['data-team-id']
        matches.append(row)
            

# Teams

In [6]:
team_code

{'ALG': '43843',
 'ANG': '43844',
 'ARG': '43922',
 'AUS': '43976',
 'AUT': '43934',
 'BEL': '43935',
 'BIH': '44037',
 'BOL': '43923',
 'BRA': '43924',
 'BUL': '43936',
 'CAN': '43899',
 'CHI': '43925',
 'CHN': '43812',
 'CIV': '43854',
 'CMR': '43849',
 'COL': '43926',
 'CRC': '43901',
 'CRO': '43938',
 'CUB': '43902',
 'CZE': '43995',
 'DEN': '43941',
 'ECU': '43927',
 'EGY': '43855',
 'ENG': '43942',
 'ESP': '43969',
 'FRA': '43946',
 'FRG': '44023',
 'GDR': '44024',
 'GER': '43948',
 'GHA': '43860',
 'GRE': '43949',
 'HAI': '43908',
 'HON': '43909',
 'HUN': '43950',
 'INH': '44031',
 'IRL': '43953',
 'IRN': '43817',
 'IRQ': '43818',
 'ISR': '43984',
 'ITA': '43954',
 'JAM': '43910',
 'JPN': '43819',
 'KOR': '43822',
 'KSA': '43835',
 'KUW': '43823',
 'MAR': '43872',
 'MEX': '43911',
 'NED': '43960',
 'NGA': '43876',
 'NIR': '43952',
 'NOR': '43961',
 'NZL': '43978',
 'PAR': '43928',
 'PER': '43929',
 'POL': '43962',
 'POR': '43963',
 'PRK': '43821',
 'ROU': '43964',
 'RSA': '43883

In [7]:
from lxml import etree
from datetime import datetime

In [12]:
teams = []
roles = {'g':'GK','d':'DF','m':'MF','f':'FW','0':'Coach',}
players = {}
ed = ['brazil2014']
for edition in ed:
    url = 'http://www.fifa.com/worldcup/archive/{:s}/teams/index.html'
    new_url = "http://www.fifa.com//worldcup/archive/edition=2014/library/teams/team={}/_players/_players_list.html"
    tree = get(url.format(edition))
    team_cols = ['No', 'Name', 'DOB', 'Position', 'Height']
    for team in tree.xpath('.//div[contains(@class, "team-qualifiedteams")]'):
        for lnk in team.xpath('.//a[contains(@class, "team")]'):
            u = lnk.get('href')
            k = re.compile(r'team=(\d+)/')
            g = k.findall(u)[0]
            subtree = get(new_url.format(g))
            country_name = [key for key,value in team_code.iteritems() if g==value]
            for elem in subtree.xpath(".//div[contains(@class,'p p-i p-i-prt-1')]"):
                age = elem.find_class('data')
                dob = age[0].attrib['data-birthdate'].replace('-','/')
                DOB = datetime.strptime(dob,'%Y/%m/%d')
                dbo = DOB.strftime('%d/%m/%Y')
                bibnum = elem.find_class('p-i-bibnum')                
                if not bibnum:
                    bibnum[0].text=' '
                players = {
                        'Edition':edition,
                        'Country':country_name[0],
                        'ID': elem.attrib['data-player-id'],
                        'Name':elem.attrib['data-player-name'],
                        'DOB':dbo,
                        'Position':roles[elem.attrib['data-player-role']],
                        'No':bibnum[0].text,
                    }
            
                teams.append(players)
            

teams = pd.DataFrame(teams)
teams
# teams.to_csv('fifa-team.csv', sep='\t', encoding='utf-8')

,Country,DOB,Edition,ID,Name,No,Position
0,ALG,09/01/1985,brazil2014,356166,Cedric SI MOHAMMED,1,GK
1,ALG,07/10/1982,brazil2014,215208,Madjid BOUGUERRA,2,DF
2,ALG,01/02/1991,brazil2014,364088,Faouzi GHOULAM,3,DF
3,ALG,01/01/1989,brazil2014,356162,Esseid BELKALEM,4,DF
4,ALG,02/09/1986,brazil2014,296827,Rafik HALLICHE,5,DF
5,ALG,09/10/1984,brazil2014,323339,Djamel MESBAH,6,DF
6,ALG,14/05/1984,brazil2014,183882,Hassan YEBDA,7,MF
7,ALG,15/03/1984,brazil2014,321682,Medhi LACEN,8,MF
8,ALG,20/04/1990,brazil2014,367261,Nabil GHILAS,9,FW
9,ALG,26/12/1989,brazil2014,319358,Sofiane FEGHOULI,10,FW


In [13]:
country_code = {}

url = "https://en.wikipedia.org/wiki/List_of_FIFA_country_codes"
tree = get(url)
for elem in tree.xpath('.//table[contains(@class,"wikitable")]'):
    for el in elem:
        k = el.text_content().strip().split('\n')
        country_code[k[0]] = k[1]

country_code['USA']= 'USA'
country_code['Korea Republic'] = 'ROK'
country_code['Korea DPR'] = 'PRK'
print country_code
   

{u'Canada': u'CAN', u'Saint Martin': u'SMN', u'Turkmenistan': u'TKM', u'West Germany': u'FRG', u'Serbia and Montenegro': u'SCG', u'East Germany': u'GDR', u'Lithuania': u'LTU', u'Cambodia': u'CAM', u'Switzerland': 'SUI', u'Ethiopia': u'ETH', u'Ceylon': u'CEY', u'Aruba': 'ARU', u'Swaziland': u'SWZ', u'Palestine': u'PLE', u'Argentina': u'ARG', u'Bolivia': u'BOL', u'Cameroon': u'CMR', u'Burkina Faso': 'BFA', u'Ghana': u'GHA', u'Saudi Arabia': 'KSA', u'South Vietnam': u'VSO', u'Martinique': u'MTQ', u'Japan': u'JPN', u'Cape Verde': u'CPV', u'Northern Mariana Islands': u'NMI', u'Slovenia': 'SVN', u'Guatemala': u'GUA', u'Bosnia and Herzegovina': u'BIH', 'USA': 'USA', u'Kuwait': u'KUW', u'Jordan': u'JOR', u'Dominica': u'DMA', u'Liberia': u'LBR', u'Maldives': u'MDV', u'Western Samoa': u'WSM', u'F.S. Micronesia': u'FSM', u'Jamaica': u'JAM', u'DR Congo': u'COD', u'Oman': u'OMA', u'Tanzania': u'TAN', u'Bonaire': u'BOE', u'Saar': u'SAA', u'Seychelles': u'SEY', u'Gabon': u'GAB', u'Niue': u'NIU', u'Sa

# Events and stats

In [15]:
# events  = []
# for match in matches:
#     url = 'http://www.fifa.com/tournaments/archive/worldcup/{Edition:s}/matches/round={RoundID:s}/match={MatchID:s}/index.html'.format(**match)
#     if match['Edition'] == 'southafrica2010':
#         url = url.replace('index.html', 'report.html')
#     tree = get(url)

#     # Summary
#     vals = tree.xpath('.//div[@class="hdTeams"]//tbody//td')
#     match['Time'] = vals[2].text
#     match['Attendance'] = vals[-1].text

#     # Stats
#     stats_table = tree.find('.//table[@class="summaryStatistics"]')
#     if stats_table is not None:
#         for row in stats_table.findall('.//tbody/tr'):
#             val1, stat, val2 = [cell.text for cell in row.findall('td')]
#             if stat in stats:
#                 stat = stats[stat]
#                 match[stat + '1'], match[stat + '2'] = val1, val2
    
#     lists = tree.xpath('.//div[contains(@class, "mrep")]//ul')
#     # G = Goal, C = Caution, E = Expulsion
#     for event, index in (('G', 0), ('C', 5), ('E', 6)):
#         for item in lists[index].findall('li'):
#             m = event_re.search(item.text)
#             name, flag, when = m.groups()
#             events.append({
#                 'MatchID': match['MatchID'],
#                 'PlayerID': player_id(match, name),
#                 'Country': flag,
#                 'Event': event,
#                 'Time': when,
#             })

#     # PG = Penalty goal, PS = Penalty save. P? = Penalty what happened?
#     if len(lists) >= 9:
#         for item in lists[8].findall('li'):
#             name = pso_re.match(item.find('.//div[@class="name"]').text)
#             name = name.group(1)
#             event = item.find('.//div[@class="conclusion"]').text
#             events.append({
#                 'MatchID': match['MatchID'],
#                 'PlayerID': player_id(match, name),
#                 'Event': 'P' + (event[0] if event is not None else '?'),
#                 'Country': '',
#                 'Time': 'PSO',
#             })

#     # Team and substitutions
#     for i, ul in enumerate(lists[1:6]):
#         name = ul.xpath('.//div[contains(@class, "bold")]')[0].text.strip()
#         team = [li.text for li in ul.findall('li//span')]

# Game-Wise Details

In [27]:
url_att = 'https://www.fifa.com/worldcup/matches/round={}/match={}/report.html'
url_stat = 'http://www.fifa.com/worldcup/matches/round={}/match={}/statistics.html'
for match in matches:
    try:
        tree_attd = get(url_att.format(match['RoundID'],match['MatchID']))
        for el in tree_attd.xpath('.//table[contains(@class,"table match-data")]//tbody//tr'):
            attendance = el.text_content()[el.text_content().index(':')+1:].lstrip('0')
        match['Attendance'] = attendance
        tree_stat =get(url_stat.format(match['RoundID'],match['MatchID']))
        for el in tree_stat.xpath('.//div[contains(@class,"fragment lb-t-stats")]'):
            k = el.find_class('stats-bar-num')
            b = el.find_class('lb-table lb-t-stats')
            count=0
            value =  []
            for row in b[0].xpath('.//tr'):
                values = row.find_class('stats-value')
                for g in values:
                    value.append(g.text_content())
            match['Fouls1'] = value[0]
            match['Fouls2'] = value[1]
            match['On Goal1']= value[2]
            match['On Goal2'] = value[3]
            match['Shots1'] = value[4]
            match['Shots2'] = value[5]
            match['Result'] = value[6] + ':' +value[7]
            match['Penalty Kicks1'] = value[8]
            match['Penalty Kicks2'] =  value[9]
        at = tree_stat.xpath('.//div[contains(@class,"statistics-wrap teamstats-wholematch")]//table[contains(@class,"table statistics-bar")]')
        element = at[0].find_class('stats-value')
        value = []
        for j in element:
            value.append(j.text_content())


    #     match['attempts1'] = value[0]
    #     attempts2 = value[1]
    #     setpieces1 = value[2]
    #     setpieces2 = value[3]
    #     match['crosses1 = value[4]
    #     crosses2 = value[5]

        match['Corner Kicks1'] = value[6]
        match['Corner Kicks2'] = value[7]
        match['Offsides1'] = value[8]
        match['Offsides2'] = value[9]
        match['Direct Free Kicks1'] = value[10]
        match['Direct Free Kicks2'] = value[11]
        caution = tree_stat.xpath('.//div[contains(@class,"statistics-wrap teamstats-wholematch")]//table[contains(@class,"table statistics-block")]')
        cards = caution[2].find_class('stats-value')
        value = []
        for j in cards:
            value.append(j.text_content())
        match['Cautions1'] = value[0]
        match['Cautions2'] = value[1]
        match['Second Cautions1'] = value[2]
        match['Second Cautions2'] =  value[3]
        match['Explusions1'] = value[4]
        match['Explusions2'] = value[5]
        match['Indirect Free Kicks1'] = '0'
        match['indirect Free Kicks2'] =  '0'
    except (IndexError):
        pass

#     print match
#         foul1,foul2,ongoal1,ongoal2,shots1,shots2,totalgoals1,totalgoals2=[],[],[],[],[],[],[],[]
       



matches = pd.DataFrame(matches)
# # matches.to_csv('fifa-match.csv')

    

        

http://www.fifa.com/worldcup/matches/round=43950100/match=43950012/statistics.html
https://www.fifa.com/worldcup/matches/round=43950100/match=43950013/report.html
http://www.fifa.com/worldcup/matches/round=43950100/match=43950013/statistics.html
https://www.fifa.com/worldcup/matches/round=43950100/match=43950014/report.html
http://www.fifa.com/worldcup/matches/round=43950100/match=43950014/statistics.html
https://www.fifa.com/worldcup/matches/round=43950100/match=43950015/report.html
http://www.fifa.com/worldcup/matches/round=43950100/match=43950015/statistics.html
https://www.fifa.com/worldcup/matches/round=43950100/match=43950016/report.html
http://www.fifa.com/worldcup/matches/round=43950100/match=43950016/statistics.html
https://www.fifa.com/worldcup/matches/round=43950100/match=43950017/report.html
http://www.fifa.com/worldcup/matches/round=43950100/match=43950017/statistics.html
https://www.fifa.com/worldcup/matches/round=43950100/match=43950020/report.html
http://www.fifa.com/wo

https://www.fifa.com/worldcup/matches/round=43950500/match=43950063/report.html
http://www.fifa.com/worldcup/matches/round=43950500/match=43950063/statistics.html
https://www.fifa.com/worldcup/matches/round=43950600/match=43950064/report.html
http://www.fifa.com/worldcup/matches/round=43950600/match=43950064/statistics.html
https://www.fifa.com/worldcup/matches/round=1014/match=4000/report.html
http://www.fifa.com/worldcup/matches/round=1014/match=4000/statistics.html
https://www.fifa.com/worldcup/matches/round=1014/match=8725/report.html
http://www.fifa.com/worldcup/matches/round=1014/match=8725/statistics.html
https://www.fifa.com/worldcup/matches/round=1014/match=8726/report.html
http://www.fifa.com/worldcup/matches/round=1014/match=8726/statistics.html
https://www.fifa.com/worldcup/matches/round=1014/match=8727/report.html
http://www.fifa.com/worldcup/matches/round=1014/match=8727/statistics.html
https://www.fifa.com/worldcup/matches/round=1014/match=8728/report.html
http://www.fif

https://www.fifa.com/worldcup/matches/round=1024/match=8780/report.html
http://www.fifa.com/worldcup/matches/round=1024/match=8780/statistics.html
https://www.fifa.com/worldcup/matches/round=1024/match=8779/report.html
http://www.fifa.com/worldcup/matches/round=1024/match=8779/statistics.html
https://www.fifa.com/worldcup/matches/round=1025/match=8781/report.html
http://www.fifa.com/worldcup/matches/round=1025/match=8781/statistics.html
https://www.fifa.com/worldcup/matches/round=1025/match=8782/report.html
http://www.fifa.com/worldcup/matches/round=1025/match=8782/statistics.html
https://www.fifa.com/worldcup/matches/round=1025/match=8784/report.html
http://www.fifa.com/worldcup/matches/round=1025/match=8784/statistics.html
https://www.fifa.com/worldcup/matches/round=1025/match=8783/report.html
http://www.fifa.com/worldcup/matches/round=1025/match=8783/statistics.html
https://www.fifa.com/worldcup/matches/round=1026/match=8785/report.html
http://www.fifa.com/worldcup/matches/round=102

http://www.fifa.com/worldcup/matches/round=796/match=3096/statistics.html
https://www.fifa.com/worldcup/matches/round=796/match=3095/report.html
http://www.fifa.com/worldcup/matches/round=796/match=3095/statistics.html
https://www.fifa.com/worldcup/matches/round=3461/match=3100/report.html
http://www.fifa.com/worldcup/matches/round=3461/match=3100/statistics.html
https://www.fifa.com/worldcup/matches/round=3461/match=3099/report.html
http://www.fifa.com/worldcup/matches/round=3461/match=3099/statistics.html
https://www.fifa.com/worldcup/matches/round=3460/match=3103/report.html
http://www.fifa.com/worldcup/matches/round=3460/match=3103/statistics.html
https://www.fifa.com/worldcup/matches/round=3459/match=3104/report.html
http://www.fifa.com/worldcup/matches/round=3459/match=3104/statistics.html
https://www.fifa.com/worldcup/matches/round=322/match=26/report.html
http://www.fifa.com/worldcup/matches/round=322/match=26/statistics.html
https://www.fifa.com/worldcup/matches/round=322/matc

https://www.fifa.com/worldcup/matches/round=308/match=459/report.html
http://www.fifa.com/worldcup/matches/round=308/match=459/statistics.html
https://www.fifa.com/worldcup/matches/round=308/match=468/report.html
http://www.fifa.com/worldcup/matches/round=308/match=468/statistics.html
https://www.fifa.com/worldcup/matches/round=308/match=439/report.html
http://www.fifa.com/worldcup/matches/round=308/match=439/statistics.html
https://www.fifa.com/worldcup/matches/round=308/match=610/report.html
http://www.fifa.com/worldcup/matches/round=308/match=610/statistics.html
https://www.fifa.com/worldcup/matches/round=308/match=395/report.html
http://www.fifa.com/worldcup/matches/round=308/match=395/statistics.html
https://www.fifa.com/worldcup/matches/round=308/match=674/report.html
http://www.fifa.com/worldcup/matches/round=308/match=674/statistics.html
https://www.fifa.com/worldcup/matches/round=308/match=379/report.html
http://www.fifa.com/worldcup/matches/round=308/match=379/statistics.html

...

https://www.fifa.com/worldcup/matches/round=263/match=2067/report.html
http://www.fifa.com/worldcup/matches/round=263/match=2067/statistics.html
https://www.fifa.com/worldcup/matches/round=263/match=2065/report.html
http://www.fifa.com/worldcup/matches/round=263/match=2065/statistics.html
https://www.fifa.com/worldcup/matches/round=263/match=2064/report.html
http://www.fifa.com/worldcup/matches/round=263/match=2064/statistics.html
https://www.fifa.com/worldcup/matches/round=263/match=1983/report.html
http://www.fifa.com/worldcup/matches/round=263/match=1983/statistics.html
https://www.fifa.com/worldcup/matches/round=263/match=1946/report.html
http://www.fifa.com/worldcup/matches/round=263/match=1946/statistics.html
https://www.fifa.com/worldcup/matches/round=263/match=2182/report.html
http://www.fifa.com/worldcup/matches/round=263/match=2182/statistics.html
https://www.fifa.com/worldcup/matches/round=264/match=1984/report.html
http://www.fifa.com/worldcup/matches/round=264/match=1984/s

http://www.fifa.com/worldcup/matches/round=238/match=1689/statistics.html
https://www.fifa.com/worldcup/matches/round=238/match=1582/report.html
http://www.fifa.com/worldcup/matches/round=238/match=1582/statistics.html
https://www.fifa.com/worldcup/matches/round=238/match=1598/report.html
http://www.fifa.com/worldcup/matches/round=238/match=1598/statistics.html
https://www.fifa.com/worldcup/matches/round=238/match=1679/report.html
http://www.fifa.com/worldcup/matches/round=238/match=1679/statistics.html
https://www.fifa.com/worldcup/matches/round=238/match=1632/report.html
http://www.fifa.com/worldcup/matches/round=238/match=1632/statistics.html
https://www.fifa.com/worldcup/matches/round=238/match=1599/report.html
http://www.fifa.com/worldcup/matches/round=238/match=1599/statistics.html
https://www.fifa.com/worldcup/matches/round=238/match=1637/report.html
http://www.fifa.com/worldcup/matches/round=238/match=1637/statistics.html
https://www.fifa.com/worldcup/matches/round=238/match=16

https://www.fifa.com/worldcup/matches/round=220/match=1418/report.html
http://www.fifa.com/worldcup/matches/round=220/match=1418/statistics.html
https://www.fifa.com/worldcup/matches/round=220/match=1423/report.html
http://www.fifa.com/worldcup/matches/round=220/match=1423/statistics.html
https://www.fifa.com/worldcup/matches/round=220/match=1388/report.html
http://www.fifa.com/worldcup/matches/round=220/match=1388/statistics.html
https://www.fifa.com/worldcup/matches/round=220/match=1332/report.html
http://www.fifa.com/worldcup/matches/round=220/match=1332/statistics.html
https://www.fifa.com/worldcup/matches/round=220/match=1391/report.html
http://www.fifa.com/worldcup/matches/round=220/match=1391/statistics.html
https://www.fifa.com/worldcup/matches/round=220/match=1324/report.html
http://www.fifa.com/worldcup/matches/round=220/match=1324/statistics.html
https://www.fifa.com/worldcup/matches/round=220/match=1406/report.html
http://www.fifa.com/worldcup/matches/round=220/match=1406/s

http://www.fifa.com/worldcup/matches/round=208/match=1230/statistics.html
https://www.fifa.com/worldcup/matches/round=208/match=1188/report.html
http://www.fifa.com/worldcup/matches/round=208/match=1188/statistics.html
https://www.fifa.com/worldcup/matches/round=208/match=1225/report.html
http://www.fifa.com/worldcup/matches/round=208/match=1225/statistics.html
https://www.fifa.com/worldcup/matches/round=208/match=1193/report.html
http://www.fifa.com/worldcup/matches/round=208/match=1193/statistics.html
https://www.fifa.com/worldcup/matches/round=208/match=1228/report.html
http://www.fifa.com/worldcup/matches/round=208/match=1228/statistics.html
https://www.fifa.com/worldcup/matches/round=208/match=1202/report.html
http://www.fifa.com/worldcup/matches/round=208/match=1202/statistics.html
https://www.fifa.com/worldcup/matches/round=208/match=1191/report.html
http://www.fifa.com/worldcup/matches/round=208/match=1191/statistics.html
https://www.fifa.com/worldcup/matches/round=208/match=11

https://www.fifa.com/worldcup/matches/round=201/match=1085/report.html
http://www.fifa.com/worldcup/matches/round=201/match=1085/statistics.html
https://www.fifa.com/worldcup/matches/round=201/match=1095/report.html
http://www.fifa.com/worldcup/matches/round=201/match=1095/statistics.html
https://www.fifa.com/worldcup/matches/round=201/match=1092/report.html
http://www.fifa.com/worldcup/matches/round=201/match=1092/statistics.html
https://www.fifa.com/worldcup/matches/round=201/match=1097/report.html
http://www.fifa.com/worldcup/matches/round=201/match=1097/statistics.html
https://www.fifa.com/worldcup/matches/round=201/match=1099/report.html
http://www.fifa.com/worldcup/matches/round=201/match=1099/statistics.html
https://www.fifa.com/worldcup/matches/round=201/match=1094/report.html
http://www.fifa.com/worldcup/matches/round=201/match=1094/statistics.html
https://www.fifa.com/worldcup/matches/round=201/match=1086/report.html
http://www.fifa.com/worldcup/matches/round=201/match=1086/s

In [25]:
# event_re = re.compile(r' *,? *(.*?) \(([A-Z]+)\) (\d+\'|PSO)')
# pso_re = re.compile(r'(.*?) \d+')
# players = teams.set_index(['Edition', 'Country', 'Name'])['ID']

# rename_player = {
#     'HONG Myungbo': 'HONG Myung Bo', # KOR, usa1994
#     'Ricardo PELAEZ': 'Ricardo PELAEZ LINARES', # MEX, france1998
#     'HWANG Bokwan': 'HWANG Bo Kwan', # KOR, italy1990
#     'HUH Jungmoo': 'HUH Jung Moo', # KOR, mexico1986
#     'JOSE TORRES': 'Jose TORRES', # POR, england1966
#     'ANTONIO SIMOES': 'Antonio SIMOES', # POR, england1966
#     'MARIO COLUNA': 'Mario COLUNA', # POR, england1966
#     'Nikita SIMONYAN': 'Nikita SIMONIAN', # URS, sweden1958
#     'KIM Taeyoung': 'KIM Tae Young', # KOR, koreajapan2002, france1998
#     # - Dobromir JECHEV is sometimes spelt at Dobromir ZECHEV of BUL? Has he played 4 cups?
# }

# stats = {
#  'Fouls Committed': 'Fouls',
#  'Cautions': 'Cautions',
#  'Direct Expulsions': 'Expulsions',
#  'Expulsions due to Second Caution': 'Second Cautions',
#  'Corner kicks': 'Corner Kicks',
#  'Direct Free Kicks to Goal': 'Direct Free Kicks',
#  'Indirect Free Kicks to Goal': 'Indirect Free Kicks',
#  'Penalty Kicks': 'Penalty Kicks',
#  'Offsides': 'Offsides',
#  'Own Goals': 'Own',
#  'Possession (%)': 'Possession',
#  'Shots': 'Shots',
#  'Shots on goal': 'On Goal'
# }

# def player_id(match, name):
#     name = rename_player.get(name, name)
#     id = players.get((match['Edition'], match['Country1'], name))
#     if id is None:
#         id = players.get((match['Edition'], match['Country2'], name))
#     if id is not None:
#         # Tunisia has duplicate players
#         # http://www.fifa.com/tournaments/archive/worldcup/koreajapan2002/teams/team=43888/index.html
#         if type(id) is not str:
#             id = id[-1]
#         return id
#     else:
#         print 'Missing', match['Edition'], match['Country1'], match['Country2'], name, hashlib.md5(url).hexdigest() + '.html'
#         return None, None
events  = []
event_con = {'event goal':'G','event yellow-card':'Y','event red-card':'E'}
event_type = ['event goal','event yellow-card','event red-card']
for match in matches:
    url = 'http://www.fifa.com/worldcup/matches/round={}/match={}/live-blog.html'
    tree = get(url.format(match['RoundID'],match['MatchID']))
    for element in tree.xpath('.//div[contains(@class,"lb-post sticky-false lb-sponsor-none evt featurephone-true")]'):
        inner_element = element.find_class('fragment lb-m-event')
        for event in inner_element:
            k =  event.xpath('.//span')
            cl = k[1].get('class')
            event = {}
            if cl in event_type:
                name_con = k[2].text_content().lstrip(')').rsplit('(')
                Name = name_con[0]
                Con = country_code[name_con[1][:name_con[1].index(')')]]
                print Name
                countries = list(teams[teams['Country']==Con]["Name"])
                lasts = []
                for el in countries:
                    last = el.split(' ')[-1].upper()
                    if Name.strip() in last:
                        player_id = teams.loc[teams["Name"]==el].iloc[-1]["ID"]
                        
                 
                    Time = k[0].text_content()
                    event['MatchID']  =  match["MatchID"]
                    event['Time'] = Time
                    event['Event'] = event_con[cl]
                    event['PlayerID'] = player_id
                    event['Player'] = Name
                    event['Country'] =  Con
                    events.append(event)
            
            
events = pd.DataFrame(events)

OSCAR 
L GUSTAVO 
LOVREN 
CORLUKA 
NEYMAR JR 
NEYMAR JR 
NOUNKEU 
O. PERALTA 
H. MORENO 
ROBBEN 
V. PERSIE 
V. PERSIE 
I. CASILLAS 
DE VRIJ 
ROBBEN 
V. PERSIE 
DE VRIJ 
DE GUZMAN 
BEAUSEJOUR 
CH. ARÃNGUIZ 
MILLIGAN 
JEDINAK 
CAHILL 
CAHILL 
VALDIVIA 
ALEXIS 
JAMES 
TEO 
SALPINGIDIS 
SOKRATIS 
C. SANCHEZ 
P. ARMERO 
M. PEREIRA 
UREÃA M. 
M. CACERES 
DUARTE O. 
W. GARGANO 
CAMPBELL J. 
D. LUGANO 
STERLING 
BALOTELLI 
STURRIDGE 
MARCHISIO 
GERVINHO 
B. WILFRIED 
MORISHIGE 
ZOKORA 
BAMBA 
YOSHIDA 
HONDA 
SEFEROVIC 
DJOUROU 
PAREDES 
MEHMEDI 
E. VALENCIA 
GARRIDO 
BENZEMA 
BONIEK 
CABAYE 
POGBA 
W. PALACIOS 
EVRA 
IBIÅ EVIÄ 
MESSI 
SPAHIÄ 
ROJO 
MÃLLER 
MÃLLER 
PEPE 
HUMMELS 
J. PEREIRA 
ANDRANIK 
MUNTARI 
BROOKS 
A. AYEW 
RABIU 
DEMPSEY 
MERTENS 
FELLAINI 
BENTALEB 
VERTONGHEN 
T SILVA 
J. VAZQUEZ 
P. AGUILAR 
RAMIRES 
J C KOO 
KERZHAKOV 
K H LEE 
SHATOV 
S Y KI 
H M SON 
MEMPHIS 
V. PERSIE 
V. PERSIE 
CAHILL 
CAHILL 
ROBBEN 
MENA 
CH. ARÃNGUIZ 
ALONSO 
VIDAL 
VARGAS 
EDUARDO 
MANDÅ

http://www.fifa.com/worldcup/matches/round=262/match=2083/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=1952/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=1954/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=1985/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=2004/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=2186/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=2097/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=1996/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=2085/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=1949/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=2175/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=1953/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/match=1987/live-blog.html
http://www.fifa.com/worldcup/matches/round=262/matc

http://www.fifa.com/worldcup/matches/round=231/match=1460/live-blog.html
http://www.fifa.com/worldcup/matches/round=231/match=1451/live-blog.html
http://www.fifa.com/worldcup/matches/round=231/match=1471/live-blog.html
http://www.fifa.com/worldcup/matches/round=231/match=1480/live-blog.html
http://www.fifa.com/worldcup/matches/round=231/match=1544/live-blog.html
http://www.fifa.com/worldcup/matches/round=231/match=1464/live-blog.html
http://www.fifa.com/worldcup/matches/round=231/match=1532/live-blog.html
http://www.fifa.com/worldcup/matches/round=232/match=1474/live-blog.html
http://www.fifa.com/worldcup/matches/round=232/match=1459/live-blog.html
http://www.fifa.com/worldcup/matches/round=232/match=1525/live-blog.html
http://www.fifa.com/worldcup/matches/round=232/match=1511/live-blog.html
http://www.fifa.com/worldcup/matches/round=514/match=1559/live-blog.html
http://www.fifa.com/worldcup/matches/round=514/match=1458/live-blog.html
http://www.fifa.com/worldcup/matches/round=3481/mat

http://www.fifa.com/worldcup/matches/round=3489/match=1149/live-blog.html
http://www.fifa.com/worldcup/matches/round=3488/match=1151/live-blog.html
http://www.fifa.com/worldcup/matches/round=3487/match=1174/live-blog.html
http://www.fifa.com/worldcup/matches/round=204/match=1104/live-blog.html
http://www.fifa.com/worldcup/matches/round=204/match=1119/live-blog.html
http://www.fifa.com/worldcup/matches/round=204/match=1133/live-blog.html
http://www.fifa.com/worldcup/matches/round=204/match=1102/live-blog.html
http://www.fifa.com/worldcup/matches/round=204/match=1108/live-blog.html
http://www.fifa.com/worldcup/matches/round=204/match=1111/live-blog.html
http://www.fifa.com/worldcup/matches/round=204/match=1135/live-blog.html
http://www.fifa.com/worldcup/matches/round=204/match=1141/live-blog.html
http://www.fifa.com/worldcup/matches/round=418/match=1143/live-blog.html
http://www.fifa.com/worldcup/matches/round=418/match=1129/live-blog.html
http://www.fifa.com/worldcup/matches/round=418/m

# Save

In [29]:
matches.to_csv('fifa-match.csv', encoding='utf-8')
# events.to_csv('event.csv',encoding='utf-8')

## Concatinating Old Frames with the New Frames

In [34]:
old_matches = pd.read_csv('fifa-match.csv')
old_teams = pd.read_csv('fifa-team.csv')
new_matches = pd.read_csv('fifa-match.csv')
new_teams = pd.read_csv('fifa-team.csv',sep='\t')
old_events = pd.read_csv('fifa-event.csv')
test = pd.concat([old_matches,new_matches],ignore_index=False,sort=True)

test_team = pd.concat([old_teams,new_teams],ignore_index=False,sort=True)

test_events = pd.concat([old_events,events],ignore_index=False,sort=True)

# Sort by MatchID (int) and Time (PSO->null)
# events_frame = pd.DataFrame(events)
# events_frame['MatchID'] = events_frame['MatchID'].astype(int)
# events_frame['Time'] = events_frame['Time'].str.replace("'", '').replace('PSO', pd.np.nan).astype(float)
# event_cols = ['MatchID', 'Time', 'Event', 'PlayerID', 'Country']
# events_frame.sort(event_cols, ascending=[False, True, True, True, True], inplace=True)
# events_frame.to_csv('fifa-event.csv', encoding='utf-8', index=False, cols=event_cols, float_format='%.0f')

# # TODO: save team_name

## Dropping Unnecessary Columns

In [ ]:
test.dropna(how='all',inplace=True)
test.fillna(method='ffill',inplace=True)
test.drop(['indirect Free Kicks2'],axis=1,inplace=True)
test.drop(['Unnamed: 0'],axis=1,inplace=True)
test_team.drop(['Unnamed: 0'],axis=1,inplace=True)

## Saving the Files

In [35]:
test.to_csv('Final-Fifa-match.csv',encoding='utf-8')
test_team.to_csv('final-fifa-team.csv',encoding='utf-8')
test_events.to_csv('final-events.csv',encoding='utf-8')